In [ ]:
#=======================================#
# Yes, this notebook is over-commented. #
#=======================================#

In [ ]:
# Make notebook span entire screen, horizontally.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import gym
import numpy as np
import tensorflow as tf

In [2]:
# TODO:
#  -> fix loss funtion
#  -> check reward processing

class PolicyAgent(object):
    def __init__(self, sess):
        self.n_actions = 4
        
        # Build graph (i.e. internal neural network).
        self._build()
        
        # Create environment within which the graph will train.
        self.sess = sess
        
        # Evaluate an tf.Operation (returned by global_variables_initializer()) that initializes TRAINABLE VARIABLES.
        self.sess.run(tf.global_variables_initializer())
        
    def _build(self):
        self.obs      = tf.placeholder(tf.float32, (None, 8))
        self.acts     = tf.placeholder(tf.int32, (None,1))
        self.target   = tf.placeholder(tf.float32, (None,1))
        self.l_rate   = tf.placeholder(tf.float32)
        self.col      = tf.placeholder(tf.int32, (None,1))
        self.training = tf.placeholder(tf.bool)
        
#         c1 = tf.layers.conv2d(self.obs, 8, [3,3], name='c1', kernel_initializer=tf.contrib.layers.xavier_initializer(),
#                               activation=tf.nn.relu)
#         c2 = tf.layers.conv2d(c1, 8, [3,3], name='c2', kernel_initializer=tf.contrib.layers.xavier_initializer(),
#                              activation=tf.nn.relu)
#         m1 = tf.layers.max_pooling2d(c2,[2,2],[2,2], name='m1')
        flat = tf.layers.flatten(self.obs, name='flat')
        
        h1 = tf.layers.dense(flat, 128, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='h1')
        drop1 = tf.layers.dropout(h1, training=self.training, name='drop1')
        h2 = tf.layers.dense(drop1,   64, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='h2')
        drop2 = tf.layers.dropout(h2, training=self.training, name='drop2')
        h3 = tf.layers.dense(drop2,   32, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='h3')
        
        drop3 = tf.layers.dropout(h3, training=self.training, name='dropout')
        
        # This needs to have linear activations to conform to the "unscaled log probabilities" requirement of `sparse_softmax_cross_entropy_loss_with_logits()`.
        self.out = tf.layers.dense(drop3, self.n_actions, kernel_initializer=tf.random_normal_initializer(), name='out')
        
        # Compute normalized probabilities associated with each action.
        self.probabilities = tf.nn.softmax(self.out)
#         indicies = tf.concat(values=[self.col, self.acts], axis=1)
        
        self.entropy = -tf.reduce_sum(self.probabilities * tf.log(tf.clip_by_value(self.probabilities,1e-10,1.0)), 1, name="entropy")
        
#         self.picked_action_prob = tf.gather_nd(self.probabilities,indicies)
        self.neg_log = tf.losses.sparse_softmax_cross_entropy(labels=self.acts, logits=self.out)
#     tf.log(tf.clip_by_value(self.picked_action_prob,1e-10,1.0))
#         self.losses = -tf.log(tf.clip_by_value(self.picked_action_prob,1e-10,1.0))
        self.loss = tf.reduce_sum((self.neg_log * self.target) - self.entropy * 0.001)
        
        # Set optimizer.
        self.train_op = tf.train.AdamOptimizer(self.l_rate).minimize(self.loss)
    
    def choose_action(self, obs, p_flag=False):
        # Compute probabilities associated with each action.
        out_vals, prob_weights = self.sess.run([self.out, self.probabilities], feed_dict={
            self.obs:      np.array(obs).reshape(-1, 8),
            self.training: False
        })
        
        if p_flag: print(prob_weights, out_vals)
        # Choose action based on computed probabilities.
        return np.random.choice(range(prob_weights.shape[1]), p=prob_weights.ravel())
    
    def train(self, act, obs, target, l_rate):
        new_acts = np.array(act).reshape(-1,1)
        _, entropy,neg_log,loss = self.sess.run([self.train_op, self.entropy, self.neg_log, self.loss], feed_dict={
            self.obs:      np.array(obs).reshape(-1, 8),
            self.acts:     np.array(act).reshape(-1,1),
            self.target:   np.array(target).reshape(-1,1),
#             self.col:      np.arange(new_acts.shape[0]).reshape(-1,1),
            self.l_rate:   l_rate,
            self.training: True
        })
        print('--------------')
        print('entropy: {}, scaled entropy:{}'.format(entropy, 0.01*entropy))
#         print('losses: {}'.format(losses))
        print('loss:  {}, neg_log:{}'.format(loss, neg_log))
#         print('probs: {}'.format(prob))
        print('target: {}'.format(target))
        print('--------------\n')
              
        return loss


In [3]:
class Critic(object):
    def __init__(self, sess):
        self.sess = sess
        self._build()
        
    def _build(self):
        self.obs     = tf.placeholder(tf.float32, (None, 8))   # The input state
        self.target  = tf.placeholder(tf.float32, (None, 1))             # The target output score
        self.lr      = tf.placeholder(tf.float32)                        # Learning Rate

        with tf.variable_scope('critic-conv'):
       
            flat = tf.layers.flatten(self.obs, name='flat')
        
        with tf.variable_scope('critic-hidden'):
            h1 = tf.layers.dense(flat, 128, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                 name='h1')
            h2 = tf.layers.dense(h1, 64, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                 name='h2')
            out = tf.layers.dense(h2, 1, kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                 name='out')
            
        self.value_estimate = tf.squeeze(out)   # [[num]] -> num
        self.loss = tf.squared_difference(self.value_estimate, self.target)
        
        self.train_op = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
        
    def predict(self, obs):
        return sess.run(self.value_estimate,feed_dict={self.obs: np.array(obs).reshape(-1, 8)})
    
    def update(self, obs, target, learning_rate=0.01):
        _, loss = self.sess.run([self.train_op, self.loss], feed_dict={self.obs:     np.array(obs).reshape(-1, 8),
                                                                       self.target:  np.array(target).reshape(-1,1),
                                                                       self.lr:      learning_rate})
        return loss
    

In [4]:
class ACHandler(object):
    def __init__(self, actor, critic, sess, env, path='./.model.ckpt'):
        self.actor = actor
        self.critic = critic
        self.env = env
        self.sess = sess
    
        self.saver = tf.train.Saver() # saves all variables
        self.path = path
        
        
        
    def init_vars(self):
        self.sess.run(tf.global_variables_initializer())
        
    def run(self, num_episodes=100, l_rate=0.001, discount=0.99, render=False, **kwargs):
        
        self.rollout(num_episodes,render,discount)
        # Close the display window
        if render: self.env.close()
            
    def play(self):
        obs_curr = self.env.reset()
        done = False
        while not done:
            self.env.render()

            # Agent chooses action based on difference frame.
            action = self.actor.choose_action(obs_curr)

            # Take action in environment.
            next_obs, reward, done, _ = self.env.step(action)
        env.close()
        
#     def train_rsample(self, batch, l_rate, num_epochs=50, mini_batch_size=100):
#         for _ in range(num_epochs):
#             indices = np.random.randint(len(batch['rew']), size=mini_batch_size)
#             self.agent.train([batch['act'][i] for i in indices],
#                              [batch['obs'][i] for i in indices],
#                              [batch['rew'][i] for i in indices],
#                              l_rate)
        
#     def train_all(self, batch, l_rate):
#         loss,neg,alt_neg = self.agent.train(batch['act'],
#                          batch['obs'],
#                          batch['rew'],
#                          l_rate)
# #         print('loss:{}     softmax:{}    alt_version:{}'.format(loss,neg,alt_neg))        
#         print('loss:{}'.format(np.mean(loss)))
    
#     def process_rewards(self, rewards, decay=0.99):
#         discounted = np.zeros_like(rewards)
#         running_reward = 0
        
#         for idx in reversed(range(len(rewards))):
#             running_reward += rewards[idx]
#             running_reward *= decay
#             discounted[idx] = running_reward
            
#         discounted -= np.mean(discounted)
#         if np.std(discounted) != 0:
#             discounted /= np.std(discounted)
#         return discounted.tolist()

    def save(self):
        self.saver.save(self.sess, self.path)
        
    def load(self):
        self.saver.restore(self.sess, self.path)
            
    def rollout(self, count, render, discount):
        batch = {'act': [], 'obs': [], 'rew': [], 'advantage':[]}
        
        for episode in range(count):
            # Stores all the stuff
            history = {'act': [], 'obs': [], 'rew': [], 'advantage':[]}
            
            obs_curr = env.reset()
            done = False
               
            while not done:
                
                if render: self.env.render()
                # Agent chooses action based on difference frame.
                action = self.actor.choose_action(obs_curr, False)
        
                # Take action in environment.
                next_obs, reward, done, _ = self.env.step(action)
                
                next_estimate = self.critic.predict(next_obs)
                td_target = reward + discount*next_estimate
                td_error = td_target - self.critic.predict(obs_curr)
                
#                 print(td_target,td_error)
#                 print('c', end=' ')
                c_loss = self.critic.update(obs_curr, td_target, 0.005)
#                 print('a', end=' ')
                a_loss = self.actor.train(action, obs_curr, td_error, 0.001)

#                 print('a_loss:{}  c_loss:{}'.format(a_loss, c_loss))
#                 print(action)
                
                history['act'].append(action)
                history['obs'].append(obs_curr)
                history['rew'].append(reward)
                history['advantage'].append(td_error)
                
                obs_curr = next_obs

            # Process rewards per episode.
#             history['rew'] = self.process_rewards(history['rew'])
            
#             self.actor.train(history['act'],history['obs'],history['advantage'],0.001)
            print(sum(history['rew']), end=' ')
            # Add episode to batch.
            for key in batch:
                batch[key].extend(history[key])
                
        return batch

In [5]:
tf.reset_default_graph()
env = gym.make('LunarLander-v2') # RGB observation space

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [6]:
sess = tf.Session()
actor = PolicyAgent(sess)
critic = Critic(sess)

In [7]:
handler = ACHandler(actor, critic, sess, env, '.models/m4.cpt')

In [8]:
handler.init_vars()

In [9]:
while(True):
    for _ in range(100):
        handler.run(num_episodes=100, l_rate=0.003, render=True, discount=0.98)
    print('Completed 100 Training Iterations\n')
    handler.save()

--------------
entropy: [0.9263641], scaled entropy:[0.00926364]
loss:  -2.3386480808258057, neg_log:2.5786080360412598
target: -0.906582837672271
--------------

--------------
entropy: [1.3752465], scaled entropy:[0.01375247]
loss:  0.22919461131095886, neg_log:1.4807556867599487
target: 0.15571093779253492
--------------

--------------
entropy: [0.900144], scaled entropy:[0.00900144]
loss:  -0.9230474829673767, neg_log:3.0427770614624023
target: -0.3030610868125678
--------------

--------------
entropy: [1.1283154], scaled entropy:[0.01128315]
loss:  0.23091772198677063, neg_log:2.620680809020996
target: 0.08854418258489549
--------------

--------------
entropy: [1.1837023], scaled entropy:[0.01183702]
loss:  -2.2821710109710693, neg_log:2.284597635269165
target: -0.9984196705567012
--------------

--------------
entropy: [1.145992], scaled entropy:[0.01145992]
loss:  1.9007242918014526, neg_log:2.5704240798950195
target: 0.7399052478341821
--------------

--------------
entropy:

--------------
entropy: [0.75108624], scaled entropy:[0.00751086]
loss:  -2.914768934249878, neg_log:3.2108850479125977
target: -0.9075435563376573
--------------

--------------
entropy: [0.82242143], scaled entropy:[0.00822421]
loss:  -1.46305251121521, neg_log:1.8257527351379395
target: -0.8008916565280231
--------------

--------------
entropy: [1.1243608], scaled entropy:[0.01124361]
loss:  -4.872114658355713, neg_log:2.313239812850952
target: -2.1057004303840188
--------------

--------------
entropy: [0.73124427], scaled entropy:[0.00731244]
loss:  -0.5475861430168152, neg_log:0.26586201786994934
target: -2.056912361668793
--------------

--------------
entropy: [1.3514977], scaled entropy:[0.01351498]
loss:  -3.5616159439086914, neg_log:1.2320177555084229
target: -2.88978334838464
--------------

--------------
entropy: [1.0475693], scaled entropy:[0.01047569]
loss:  -1.1480540037155151, neg_log:0.45977964997291565
target: -2.4946869644977028
--------------

--------------
entr

--------------
entropy: [1.0453779], scaled entropy:[0.01045378]
loss:  -5.7150797843933105, neg_log:2.9043314456939697
target: -1.9674181986522115
--------------

--------------
entropy: [1.3331797], scaled entropy:[0.0133318]
loss:  0.22456307709217072, neg_log:1.2362284660339355
target: 0.1827301845206648
--------------

--------------
entropy: [1.0924548], scaled entropy:[0.01092455]
loss:  1.8706469535827637, neg_log:1.1515181064605713
target: 1.6254536768358179
--------------

--------------
entropy: [1.153835], scaled entropy:[0.01153835]
loss:  -0.04461701959371567, neg_log:0.6234603524208069
target: -0.06971283226165781
--------------

--------------
entropy: [1.3553095], scaled entropy:[0.01355309]
loss:  2.869722843170166, neg_log:1.2189738750457764
target: 2.3553238106527346
--------------

--------------
entropy: [0.8121864], scaled entropy:[0.00812186]
loss:  0.8792884945869446, neg_log:0.560522198677063
target: 1.5701441317064848
--------------

--------------
entropy: [

--------------
entropy: [1.3750573], scaled entropy:[0.01375057]
loss:  -9.900851249694824, neg_log:1.152634859085083
target: -8.588562070351184
--------------

--------------
entropy: [0.94736195], scaled entropy:[0.00947362]
loss:  -9.057538032531738, neg_log:2.4152371883392334
target: -3.749772674469849
--------------

--------------
entropy: [0.15122992], scaled entropy:[0.0015123]
loss:  -32.46463394165039, neg_log:5.137932300567627
target: -6.31858882475564
--------------

--------------
entropy: [0.8611796], scaled entropy:[0.0086118]
loss:  -1.5287277698516846, neg_log:0.3760360777378082
target: -4.063084936591917
--------------

--------------
entropy: [0.89378685], scaled entropy:[0.00893787]
loss:  -2.4496240615844727, neg_log:0.39313429594039917
target: -6.228737140068123
--------------

--------------
entropy: [0.00212717], scaled entropy:[2.1271666e-05]
loss:  -38.78548812866211, neg_log:8.749876022338867
target: -4.432689550150368
--------------

--------------
entropy: 

--------------
entropy: [0.33280876], scaled entropy:[0.00332809]
loss:  0.6272532343864441, neg_log:0.076586052775383
target: 8.194522329598641
--------------

--------------
entropy: [1.0745646], scaled entropy:[0.01074565]
loss:  5.274144649505615, neg_log:0.9190122485160828
target: 5.740096807818702
--------------

--------------
entropy: [0.00953962], scaled entropy:[9.53962e-05]
loss:  0.008766904473304749, neg_log:0.0011470888275653124
target: 7.651059127838607
--------------

--------------
entropy: [0.45520803], scaled entropy:[0.00455208]
loss:  1.057652235031128, neg_log:0.14660295844078064
target: 7.2175045528664725
--------------

--------------
entropy: [0.04870928], scaled entropy:[0.00048709]
loss:  0.0505119264125824, neg_log:0.007383794989436865
target: 6.847513384616946
--------------

--------------
entropy: [1.1073248], scaled entropy:[0.01107325]
loss:  7.408166885375977, neg_log:1.0999908447265625
target: 6.735759622061096
--------------

--------------
entropy: 

...

--------------
entropy: [0.16878939], scaled entropy:[0.00168789]
loss:  0.05126572027802467, neg_log:0.033539146184921265
target: 1.533566424630319
--------------

--------------
entropy: [0.74194825], scaled entropy:[0.00741948]
loss:  1.2277109622955322, neg_log:0.6613606214523315
target: 1.8574631237261663
--------------

--------------
entropy: [1.1664591], scaled entropy:[0.01166459]
loss:  0.2810647189617157, neg_log:0.6483649611473083
target: 0.4352967950093998
--------------

--------------
entropy: [1.0724775], scaled entropy:[0.01072477]
loss:  -1.8464840650558472, neg_log:1.0850712060928345
target: -1.700728591108085
--------------

--------------
entropy: [1.2515239], scaled entropy:[0.01251524]
loss:  2.099722385406494, neg_log:1.6061238050460815
target: 1.30810200298788
--------------

--------------
entropy: [0.13309154], scaled entropy:[0.00133092]
loss:  11.309894561767578, neg_log:6.063474655151367
target: 1.8652716700602134
--------------

--------------
entropy: [0

--------------
entropy: [0.03136136], scaled entropy:[0.00031361]
loss:  9.0328369140625, neg_log:6.386185169219971
target: 1.4144388975444766
--------------

--------------
entropy: [0.03136771], scaled entropy:[0.00031368]
loss:  17.81344223022461, neg_log:6.894824504852295
target: 2.58360052578837
--------------

--------------
entropy: [1.2654426], scaled entropy:[0.01265443]
loss:  -1.72939133644104, neg_log:1.2541919946670532
target: -1.3778799000784758
--------------

--------------
entropy: [0.33045042], scaled entropy:[0.0033045]
loss:  0.08277468383312225, neg_log:0.07809866219758987
target: 1.0641044367880514
--------------

--------------
entropy: [1.1065035], scaled entropy:[0.01106503]
loss:  0.6559202671051025, neg_log:0.6016234159469604
target: 1.0920898043633294
--------------

--------------
entropy: [0.8261537], scaled entropy:[0.00826154]
loss:  1.1428519487380981, neg_log:4.030491828918457
target: 0.2837564509764334
--------------

--------------
entropy: [0.997861

--------------
entropy: [0.86633736], scaled entropy:[0.00866337]
loss:  1.1238154172897339, neg_log:2.0693717002868652
target: 0.543489466475938
--------------

--------------
entropy: [1.1394316], scaled entropy:[0.01139432]
loss:  7.0077900886535645, neg_log:2.329052448272705
target: 3.0093482726765926
--------------

--------------
entropy: [1.1537243], scaled entropy:[0.01153724]
loss:  1.9518892765045166, neg_log:0.5854818820953369
target: 3.3357873527418525
--------------

--------------
entropy: [0.15188074], scaled entropy:[0.00151881]
loss:  -29.68320083618164, neg_log:7.542759895324707
target: -3.9353035196548447
--------------

--------------
entropy: [0.16616556], scaled entropy:[0.00166166]
loss:  0.024448024109005928, neg_log:0.03329034522175789
target: 0.7393792242530299
--------------

--------------
entropy: [1.0188674], scaled entropy:[0.01018867]
loss:  4.4485697746276855, neg_log:3.004211187362671
target: 1.4811173043846386
--------------

--------------
entropy: [

--------------
entropy: [0.33552015], scaled entropy:[0.0033552]
loss:  -10.274157524108887, neg_log:3.6649460792541504
target: -2.8032667575983368
--------------

--------------
entropy: [1.2021871], scaled entropy:[0.01202187]
loss:  -0.6101889610290527, neg_log:1.9326386451721191
target: -0.3151063909264913
--------------

--------------
entropy: [1.1509763], scaled entropy:[0.01150976]
loss:  -1.302963137626648, neg_log:0.6994925737380981
target: -1.8610808209766212
--------------



KeyboardInterrupt: 

In [ ]:
handler.load()

In [ ]:
handler.save()

In [ ]:
handler.play()